Setup necessary modules

In [26]:
%use dataframe
%use kandy

Setup file columns and folders

In [27]:
import java.nio.file.Paths
import java.nio.file.Files
import java.io.BufferedInputStream
import org.apache.commons.csv.CSVFormat
import kotlin.io.path.inputStream

val date by column<Int>("DATE")
val count by column<Int>("count")
val tavg by column<Double>("TAVG")
val tmin by column<Double>("TMIN")
val tmax by column<Double>("TMAX")
val average by column<Double>()
val max by column<Double>()
val min by column<Double>()

val name by column<String>("NAME")
val latitude by column<Double>("LATITUDE")
val longitude by column<Double>("LONGITUDE")
val elevation by column<Double>("ELEVATION")
val snow by column<Double>("SNOW")

//hacky
val projectFolder = Paths.get(System.getProperty("user.dir").replace("/src/main/kotlin/org/data/first/","")).normalize()
println(projectFolder)
val rawDataFolder = projectFolder.resolve("raw-data")
println(rawDataFolder)
val rawDataYearlyDataFolder = rawDataFolder.resolve("yearly-weather")
println(rawDataYearlyDataFolder)
val refinedDataFolder = projectFolder.resolve("refined-data")
println(refinedDataFolder)
val refinedDataYearlyDataFolder = refinedDataFolder.resolve("yearly-data")
refinedDataYearlyDataFolder.toFile().mkdirs()
println(refinedDataYearlyDataFolder)
val presentableDataFolder = projectFolder.resolve("presentable-data")
presentableDataFolder.toFile().mkdirs()
println(presentableDataFolder)

val highElevationStart = 2000.0

/Users/jansrnicek/IdeaProjects/article.1
/Users/jansrnicek/IdeaProjects/article.1/raw-data
/Users/jansrnicek/IdeaProjects/article.1/raw-data/yearly-weather
/Users/jansrnicek/IdeaProjects/article.1/refined-data
/Users/jansrnicek/IdeaProjects/article.1/refined-data/yearly-data
/Users/jansrnicek/IdeaProjects/article.1/presentable-data


Count all raw data samples

In [29]:

Files.list(rawDataYearlyDataFolder)
    .map { Files.readAllLines(it).count() }
   .toList()
    .sum()

2658574

Refine data to only the ones that have minimum,maximum and average temparatures available

In [30]:

var counter = 0
Files.list(rawDataYearlyDataFolder)
    .toList()   
    .chunked(1000)
    .forEach { chunk ->
        val outFile = refinedDataYearlyDataFolder.resolve("yearly_data_part_$counter.csv")
        Files.writeString(outFile, chunk.map {
            DataFrame.read(it.toFile())
                .filter { it.containsKey("TAVG") }
                .filter { it.get("TAVG") != null }
                .filter { !tavg.getValue(it).isNaN() }
                .filter { it.containsKey("TMIN") }
                .filter { it.get("TMIN") != null }
                .filter { !tmin.getValue(it).isNaN() }
                .filter { it.containsKey("TMAX") }
                .filter { it.get("TMAX") != null }
                .filter { !tmax.getValue(it).isNaN() }
        }
            .toList()
            .concat()
            .toCsv(CSVFormat.DEFAULT))
        counter += 1
    }


Count data samples after refinement

In [31]:

Files.list(refinedDataYearlyDataFolder)
    .map { Files.readAllLines(it).count() }
    .toList()
    .sum()

868331

Aggregate all files to a single file with count of data samples, min,max and average temparatures

In [32]:


Files.writeString(presentableDataFolder.resolve("basic-temparatures.csv"), Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map { DataFrame.readCSV(BufferedInputStream(it.inputStream())) }
            .toList()
            .concat()
            .groupBy(date)
            .sortBy(date)
            .aggregate {
                count() into count
                mean(tavg) into average
                mean(tmin) into min
                mean(tmax) into max
            }
    }
    .toList()
    .concat()
    .convert(count).with{it.toInt()}
    .groupBy(date)
    .sortBy(date)
    .aggregate {
        sum(count) into count
        mean(average) into average
        mean(min) into min
        mean(max) into max
    }
    .toCsv())

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/basic-temparatures.csv

Plot min/max temparatures

In [33]:

DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("basic-temparatures.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(min) {
                scale = continuous(0.0..20.0)
                axis {
                    name = "Temperatures"
                }
            }
            color(min) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }
        line {
            x(date)
            y(max) { scale = continuous(0.0..20.0) }
            color(max) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="tc0Bby"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1763.0,1764.0,1765.0,1766.0,1767.0,1768.0,1769.0,1770.0,1771.0,1772.0,1773.0,1775.0,1776.0,1777.0,1778.0,1779.0,1780.0,1781.0,1782.0,1783.0,1784.0,1785.0,1786.0,1787.0,1788.0,1789.0,1790.0,1791.0,1792.0,1793.0,1794.0,1795.0,1796.0,1797.0,1798.0,1799.0,1800.0,1801.0,1802.0,1803.0,1804.0,1805.0,1806.0,1807.0,1808.0,1809.0,1810.0,1811.0,1812.0,1813.0,1814.0,1815.0,1816.0,1817.0,1818.0,1819.0,1820.0,1821.0,1822.0,1823.0,1824.0,1825.0,1826.0,1827.0,1828.0,1829.0,1830.0,1831.0,1832.0,1833.0,1834.0,1835.0,1836.0,1837.0,1838.0,1839.0,1840.0,1841.0,1842.0,1843.0,1844.0,1845.0,1846.0,1847.0,1848.0,1849.0,1850.0,1851.0,1852.0,1853.0,1854.0,1855.0,1856.0,1857.0,1858.0,1859.0,1860.0,1861.0,1862.0,1863.0,1864.0,1865.0,1866.0,1867.0,1868.0,1869.0,1870.0,1871.0,1872.0,1873.0,1874.0,1875.0,1876.0,1877.0,1878.0,1879.0,1880.0,1881.0,1882.0,1883.0,1884.0,1885.0,1886.0,1887.0,1888.0,1889.0,1890.0,1891.0,1892.0,1893.0,1894.0,1895.0,1896.0,1897.0,1898.0,1899.0,1900.0,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"min":[9.98,10.6,10.49,10.14,9.14,9.85,10.67,10.63,10.78,12.15,10.54,9.26,7.865,7.76,6.65,6.82,5.34,9.095,5.77,8.965,7.609999999999999,7.41,7.16,9.105,6.44,8.530000000000001,8.915,8.02,8.955,9.26,8.49,8.355,8.74,9.290000000000001,8.84,6.970000000000001,6.99,9.385,9.145,6.13,6.54,7.109999999999999,9.335,7.41,7.55,8.43,6.03,9.565,7.185,6.32,5.08,6.646666666666667,6.150000000000001,6.82,7.816666666666666,7.813333333333333,6.9433333333333325,8.31,9.325,6.986666666666667,6.8575,7.25,7.15,6.906666666666666,7.5525,5.21,6.41,6.380000000000001,6.175,6.282,7.142,5.984,6.048,5.55,4.51,6.042,4.894,5.67,5.194000000000001,5.865,5.189166666666668,5.168000000000001,6.040000000000001,5.253333333333333,5.586666666666666,5.423333333333333,5.187500000000001,5.484166666666667,6.003333333333334,5.043333333333333,5.3500000000000005,4.615,6.126428571428571,6.367500000000001,5.624375,6.7645,6.413333333333333,5.993181818181819,6.114545454545453,7.411874999999999,4.491666666666666,4.853125,5.697100000000001,4.37925,6.159363636363637,3.749538461538462,4.120416666666666,3.983622222222222,5.13303125,4.278057422969188,4.618637254901961,3.340944911297852,4.1968870214752565,4.663696078431372,5.236458216619983,3.6592157940099113,4.347116013071894,4.178108458534929,3.829633151162563,2.416987894458483,2.9728830007939,2.295137289181407,2.4914678907693615,2.514194255091314,2.5832145791514987,3.6066202463434296,3.1812021538251507,3.367292437509583,3.3146635154938338,3.7240239293552375,4.638244325390847,3.825787865848067,4.792846951950129,4.641075952693771,4.906794813159521,4.2005515180736195,5.07992859529347,4.523805911047404,4.623394259154977,4.600978239516458,3.988309945849023,4.552132817308918,5.163075216429972,4.72845583877757,5.076465049010663,4.621691572817746,4.8218

Plot weather stations

In [34]:

DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("basic-temparatures.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(count) {
                axis {
                    name = "Weather stations"
                }
            }
            color(count) {
                scale = continuous(Color.RED..Color.GREEN)
                legend {
                    name = "Weather stations"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CHBkNt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1763.0,1764.0,1765.0,1766.0,1767.0,1768.0,1769.0,1770.0,1771.0,1772.0,1773.0,1775.0,1776.0,1777.0,1778.0,1779.0,1780.0,1781.0,1782.0,1783.0,1784.0,1785.0,1786.0,1787.0,1788.0,1789.0,1790.0,1791.0,1792.0,1793.0,1794.0,1795.0,1796.0,1797.0,1798.0,1799.0,1800.0,1801.0,1802.0,1803.0,1804.0,1805.0,1806.0,1807.0,1808.0,1809.0,1810.0,1811.0,1812.0,1813.0,1814.0,1815.0,1816.0,1817.0,1818.0,1819.0,1820.0,1821.0,1822.0,1823.0,1824.0,1825.0,1826.0,1827.0,1828.0,1829.0,1830.0,1831.0,1832.0,1833.0,1834.0,1835.0,1836.0,1837.0,1838.0,1839.0,1840.0,1841.0,1842.0,1843.0,1844.0,1845.0,1846.0,1847.0,1848.0,1849.0,1850.0,1851.0,1852.0,1853.0,1854.0,1855.0,1856.0,1857.0,1858.0,1859.0,1860.0,1861.0,1862.0,1863.0,1864.0,1865.0,1866.0,1867.0,1868.0,1869.0,1870.0,1871.0,1872.0,1873.0,1874.0,1875.0,1876.0,1877.0,1878.0,1879.0,1880.0,1881.0,1882.0,1883.0,1884.0,1885.0,1886.0,1887.0,1888.0,1889.0,1890.0,1891.0,1892.0,1893.0,1894.0,1895.0,1896.0,1897.0,1898.0,1899.0,1900.0,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"count":[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,5.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,9.0,10.0,10.0,12.0,11.0,12.0,12.0,10.0,13.0,13.0,19.0,20.0,20.0,25.0,26.0,31.0,41.0,50.0,61.0,74.0,79.0,89.0,103.0,116.0,118.0,128.0,146.0,150.0,173.0,178.0,200.0,214.0,210.0,232.0,251.0,263.0,289.0,648.0,866.0,998.0,1076.0,1214.0,1328.0,1390.0,1431.0,1255.0,1402.0,1536.0,1607.0,1729.0,1740.0,1734.0,1819.0,1932.0,2031.0,2129.0,2202.0,2277.0,2374.0,2446.0,2418.0,2455.0,2359.0,2355.0,2366.0,2489.0,2510.0,2565.0,2625.0,2716.0,2770.0,2796.0,2903.0,2947.0,3123.0,3285.0,3336.0,3445.0,3468.0,3487.0,3587.0,3689.0,3876.0,4069.0,4147.0,4198.0,4278.0,4301.0,4525.0,4587.0,4682.0,4757.0,4787.0,5498.0,5787.0,6451.0,6795.0,7075.0,7294.0,7524.0,7667.0,7957.0,8183.0,8313.0,8609.0,9194.0,9497.0,9855.0,9817.0,10193.0,10452.0,10617.0,10359.0,10315.0,10815.0,10868.0,10775.0,10424.0,10691.0,10725.0,10630.0,10668.0,10633.0,10340.0,10338.0,10648.0,10205.0,10402.0,10087.0,9709.0,9643.0,9781.0,9728.0,9819.0,9654.0,9940.0,9712.0,9340.0,9696.0,9923.0,10059.0,10116.0,10076.0,10143.0,10189.0,10257.0,10315.0,10461.0,10407.0,10583.0,10630.0,10426.0,10227.0,10530.0,10771.0,11026.0,10313.0,9353.0,9371.0,9359.0,9104.0,8875.0,8788.0,8331.0,8510.0,8422.0,8218.0,7985.0]
},
"ggsize":{
"width":1500.0,
"height":800.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Year",
"format":"{d}",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Weather stations",
"limits":[null,null]
},{
"aesthetic":"color"

Put data for all stations into GPX format so we can feed it int omap

In [35]:
data class GpxEntry(val name: String, val latitude: Double, val longitude: Double) {
    override fun toString(): String {
        return """<wpt lat="${latitude}" lon="${longitude}">
		            <name>${name}</name>
	              </wpt>"""
    }
}

data class GpxTemplate(val points: List<GpxEntry>) {
    override fun toString(): String {
        return """<?xml version="1.0" encoding="UTF-8"?>
                    <gpx version="1.1" xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd">
	                    ${points.map { it.toString() }.joinToString("\n")}
                    </gpx>"""
    }
}

In [36]:


val points = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map { DataFrame.readCSV(BufferedInputStream(it.inputStream())) }
            .toList()
            .concat()
            .groupBy(name, latitude, longitude)
            .count()
    }
    .toList()
    .concat()
    .groupBy(name, latitude, longitude)
    .count()
    .map {
        GpxEntry(
            name = it.getValue(name), latitude = it.getValue(latitude), longitude = it.getValue(longitude)
        )
    }
    .toList()

Files.writeString(presentableDataFolder.resolve("gpx-visualization.gpx"),GpxTemplate(points).toString())


/Users/jansrnicek/IdeaProjects/article.1/presentable-data/gpx-visualization.gpx

In [37]:
Files.writeString(presentableDataFolder.resolve("duplicate-values.csv"),Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { date.getValue(it) >= 1900 }
        }
            .toList()
            .concat()
            .groupBy(name,date)
            .aggregate {
                count() into count
            }
    }
    .toList()
    .concat()
    .convert(count).with { it.toInt() }
    .groupBy(name,date)
    .aggregate {
        sum(count) into count
    }
    .filter { count.getValue(it) > 1 }//duplicates
    .sortByDesc(date)
    .toCsv())

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/duplicate-values.csv

In [38]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("duplicate-values.csv").inputStream()))
    .take(10)

NAME,DATE,count
SAVANNAH HILTON HEAD INTERNATIONAL AI...,2023,2
"MARYBOROUGH, AS",2023,2
"TERRE HAUTE HULMAN REGIONAL AIRPORT, ...",2023,2
"APALACHICOLA AIRPORT, FL US",2023,2
"VICTORIA REGIONAL AIRPORT, TX US",2023,2
"SAN JOSE, CA US",2023,2
"CORNER BROOK, NL CA",2023,2
"YORKTON, SK CA",2023,2
"BAIE COMEAU, QC CA",2023,2
"TROIS RIVIERES, QC CA",2023,2


In [39]:
val limitYear = 1980
val expectedValueCount = 2023 - limitYear + 1
val error = 1 //one year missing
val solidDataFileName = "solid-data-since-$limitYear.csv"

In [40]:

val potentialCandidates = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { date.getValue(it) >= limitYear }
        }
            .toList()
            .concat()
            .groupBy(name)
            .aggregate {
                count() into count
            }
    }
    .toList()
    .concat()
    .convert(count).with { it.toInt() }
    .groupBy(name)
    .aggregate {
        sum(count) into count
    }
    .filter { count.getValue(it) >= (expectedValueCount - error) }//duplicates
    .sortByDesc(count)
    .select(name)
    .convert(name).with { it.toString() }
    .map { name.getValue(it) }

val weatherStations = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { potentialCandidates.contains(name.getValue(it)) }
                .filter { date.getValue(it) >= (expectedValueCount - error) }
        }
            .toList()
            .concat()
    }
    .toList()
    .concat()
    .distinctBy(name, date)
    .groupBy(name)
    .sortBy(name, date)
    .aggregate {
        count() into count
    }
    .convert(count).with { it.toInt() }
    .filter { count.getValue(it) >= (expectedValueCount - error) }
    .select(name)
    .convert(name).with { it.toString() }
    .map { name.getValue(it) }


Files.writeString(presentableDataFolder.resolve(solidDataFileName),
    Files.list(refinedDataYearlyDataFolder)
        .toList()
        .chunked(5)
        .map { chunk ->
            chunk.map {
                DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                    .filter { weatherStations.contains(name.getValue(it)) }
                    .filter { date.getValue(it) >= limitYear }
            }
                .toList()
                .concat()
        }
        .toList()
        .concat()
        .toCsv())

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/solid-data-since-1980.csv

In [41]:
val presentableFileName = "yearly-temparatures-since-$limitYear.csv"

Files.writeString(presentableDataFolder.resolve("yearly-temparatures-since-$limitYear.csv"), DataFrame.readCSV(
    BufferedInputStream(
        presentableDataFolder.resolve(solidDataFileName).inputStream()
    )
)
    .groupBy(date)
    .aggregate {
        mean(tmin) into min
        mean(tmax) into max
        mean(tavg) into average
    }
    .toCsv())

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/yearly-temparatures-since-1980.csv

Plot solid data since 1900

In [42]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("yearly-temparatures-since-$limitYear.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(min) {
                scale = continuous(0.0..20.0)
                axis {
                    name = "Temperatures"
                }
            }
            color(min) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }
        line {
            x(date)
            y(max) { scale = continuous(0.0..20.0) }
            color(max) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="OHjfRp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2003.0,2005.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,2004.0,2006.0,1980.0,1981.0,2002.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"min":[6.548290993071591,6.14433940774487,6.655406320541747,6.6558856502242145,5.837714604236344,6.3229524886877755,7.122293785310735,6.709393258426957,6.844480812641083,6.786583990980844,6.535182648401827,6.758051801801796,6.366459489456166,6.6458050383351654,6.446433333333339,5.6038901345291405,6.264282433983929,6.043659378596084,6.229624573378845,6.361895953757221,6.925563298490131,6.6008597285067925,6.672970521541942,7.106845039018944,6.080901916572715,6.4979185022026416,6.9138926174496635,6.893513513513509,6.478671171171168,6.4256719817767705,6.113737259343152,6.642888377445351,6.5354523522316095,5.729721871049304,6.214269377382467,6.561178707224338,6.695019011406847,6.4771994884910455,6.44512886597938,6.312162849872787,6.799683544303798,6.3532065906210455,6.336269230769228,6.925086782376503],
"max":[16.8775635103926,16.562619589977256,17.4358690744921,17.318475336322873,16.485484949832774,16.95130090497737,17.498,17.61116853932585,17.54460496613995,17.4129425028185,17.442351598173513,17.53373873873875,17.035083240843505,17.20508214676889,16.873333333333346,16.207791479820646,16.775706084959804,16.612474108170304,17.076541524459625,17.134381502890175,17.567108013937272,17.187205882352952,17.14772108843536,18.068104793756962,16.741747463359644,17.093381057268722,17.685279642058173,17.603727477477477,17.178209459459456,17.05071753986331,16.601936579841457,17.473855005753737,17.217901085645334,15.671554993678882,16.249656925031765,16.70548795944235,16.714055766793415,16.567992327365733,16.660876288659807,16.46267175572517,17.051911392405053,16.46626108998735,16.912320512820493,17.15941255006674]
},
"ggsize":{
"width":1500.0,
"height":800.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Year",
"format":"{d}",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Temperatures",
"limits":[0.0,20.0]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#5470c6",
"name":"Temparature in Celsius",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[0.0,20.0]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#5470c6",
"name":"Temparature in Celsius",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"DATE",
"y":"min",
"color":"min"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"DATE",
"y":"max",
"color":"max"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("OHjfRp");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 
 
 
 
 
 
 1985 
 
 
 
 
 
 
 
 
 1990 
 
 
 
 
 
 
 
 
 1995 
 
 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 
 
 2005 
 
 
 
 
 
 
 
 
 2010 
 
 
 
 
 
 
 
 
 2015 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2025 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

In [43]:
import org.jetbrains.letsPlot.Geom
import org.jetbrains.letsPlot.annotations.layerLabels
import org.jetbrains.letsPlot.scale.guideLegend

val group by column<Int>()
val groupName by column<String>()

DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve(solidDataFileName).inputStream()))
    .add(group) { elevation.getValue(it) / 500 }
    .convert(group).to<Int>()
    .add(groupName) {
        val groupValue = group.getValue(it)
        val left = "${groupValue * 500}"
        val right = "${((groupValue + 1) * 500) - 1}"
        "$left - $right"
    }
    .groupBy(group,groupName,date)
    .aggregate {
        count() into count
    }
    .sortBy(group)
    .groupBy(groupName)
    .aggregate {
        mean(count) into count
    }
    .convert(count).to<Int>()
    .plot {
        x(groupName){
            axis{
                name = "Elevation group"
                breaks(format = "{} m")
            }
        }
        bars {
            y(count){
                axis.name = "Number of stations"
            }
            
        }
        layout.size = Pair(1000,450)
    }
   

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rujXLX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"groupName":["0 - 499","500 - 999","1000 - 1499","1500 - 1999","2000 - 2499","2500 - 2999","3000 - 3499"],
"count":[505.0,224.0,61.0,32.0,27.0,7.0,1.0]
},
"ggsize":{
"width":1000.0,
"height":450.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Elevation group",
"format":"{} m"
},{
"aesthetic":"y",
"name":"Number of stations",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"groupName",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("rujXLX");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 - 499 m 
 
 
 
 
 
 
 
 
 500 - 999 m 
 
 
 
 
 
 
 
 
 1000 - 1499 m 
 
 
 
 
 
 
 
 
 1500 - 1999 m 
 
 
 
 
 
 
 
 
 2000 - 2499 m 
 
 
 
 
 
 
 
 
 2500 - 2999 m 
 
 
 
 
 
 
 
 
 3000 - 3499 m 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 450 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Number of stations 
 
 
 
 
 Elevation group

In [44]:
val group by column<Int>()
val groupName by column<String>()

Files.writeString(presentableDataFolder.resolve("elevation-groups-$limitYear"), DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve(solidDataFileName).inputStream()))
    .add(group) { elevation.getValue(it) / 500 }
    .convert(group).to<Int>()
    .add(groupName) {
        val groupValue = group.getValue(it)
        val left = "${groupValue * 500}"
        val right = "${((groupValue + 1) * 500) - 1}"
        "$left - $right"
    }
    .groupBy(groupName,group, date)
    .aggregate {
        count() into count
        mean(tmin) into min
        mean(tmax) into max
    }
    .toCsv())
    

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/elevation-groups-1980

In [45]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("elevation-groups-$limitYear").inputStream()))
    .sortBy(group)
    .groupBy(groupName)
    .filter { 
        groupName.getValue(it) != "2500 - 2999" &&groupName.getValue(it) != "3000 - 3499"  
    }
    .plot {
        line {
            x(date) {
                axis.name = "Year"
            }
            y(min) {
                axis.name = "Minimum Temparature"
            }
            color(groupName) {
                legend {
                    name = "Elevation group"
                }
            }
        }
        layout.size = Pair(1000, 450)
    }


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xsl2D6"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2003.0,2005.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,2002.0,2004.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1999.0,2000.0,2001.0,2004.0,2005.0,2006.0,1981.0,1998.0,2002.0,2003.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2013.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1990.0,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,1988.0,1989.0,1994.0,1998.0,1999.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2023.0,2022.0],
"groupName":["0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999"

In [46]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("elevation-groups-$limitYear").inputStream()))
    .sortBy(group)
    .groupBy(groupName)
    .filter {
        groupName.getValue(it) != "2500 - 2999" &&groupName.getValue(it) != "3000 - 3499"
    }
    .plot {
    line {
        x(date) {
            axis.name = "Year"
        }
        y(max) {
            axis.name = "Maximum Temparature"
        }
        color(groupName) {
            legend {
                name = "Elevation group"
            }
        }
    }
    layout.size = Pair(1000, 450)
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="B9CPd9"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2003.0,2005.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,2002.0,2004.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1999.0,2000.0,2001.0,2004.0,2005.0,2006.0,1981.0,1998.0,2002.0,2003.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2013.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1990.0,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,1988.0,1989.0,1994.0,1998.0,1999.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2023.0,2022.0],
"groupName":["0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999"

In [47]:
val snowFallData =
    DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve(solidDataFileName).inputStream()))
        .filter { it.get("SNOW") != null }
        .filter { !snow.getValue(it).isNaN() }
        .add(group) { elevation.getValue(it) / 500 }
        .convert(group).to<Int>()
        .add(groupName) {
            val groupValue = group.getValue(it)
            val left = "${groupValue * 500}"
            val right = "${((groupValue + 1) * 500) - 1}"
            "$left - $right"
        }
        .groupBy(groupName, group, date)
        .aggregate {
            count() into count
            mean(snow) into snow
        }
        .sortBy(group)
        .groupBy(groupName)
        .filter {
            groupName.getValue(it) != "2500 - 2999" && groupName.getValue(it) != "3000 - 3499"
        }

In [48]:
snowFallData
    .plot {
        line {
            x(date) {
                axis.name = "Year"
            }
            y(snow) {
                axis.name = "Average snowfall"
                scale = continuous(0.0..3000.0)
            }
            color(groupName) {
                legend {
                    name = "Average snowfall"
                }
            }
        }
        layout.size = Pair(1000, 450)
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VTYvDn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1981.0,1983.0,1984.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1982.0,1985.0,2001.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2000.0,2001.0,2002.0,2003.0,2004.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1997.0,1998.0,1999.0,2000.0,2001.0,2003.0,2004.0,2002.0,1996.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2011.0,2012.0],
"groupName":["0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999","1500 - 1999"

In [49]:
val prcp by column<Double>("PRCP")

Files.writeString(presentableDataFolder.resolve("precipitation-since-$limitYear.csv"),DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve(solidDataFileName).inputStream()))
    .filter { it.get("PRCP")!=null }
    .add(group) { elevation.getValue(it) / 500 }
    .convert(group).to<Int>()
    .add(groupName) {
        val groupValue = group.getValue(it)
        val left = "${groupValue * 500}"
        val right = "${((groupValue + 1) * 500) - 1}"
        "$left - $right"
    }
    .groupBy(groupName, group, date)
    .aggregate {
        mean(prcp) into prcp
    }.toCsv())
    

/Users/jansrnicek/IdeaProjects/article.1/presentable-data/precipitation-since-1980.csv

In [50]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("precipitation-since-$limitYear.csv").inputStream()))
    .plot {
        line {
            x(date) {
                axis.name = "Year"
            }
            y(prcp) {
                axis.name = "Average rain"
                scale = continuous(0.0..3000.0)
            }
            color(groupName) {
                legend {
                    name = "Average rain"
                }
            }
        }
        layout.size = Pair(1000, 450)
    }
    

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="e0Rpk5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2003.0,2005.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1999.0,2000.0,2001.0,2004.0,2005.0,2006.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,2002.0,2004.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1981.0,1998.0,2002.0,2003.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2014.0,2015.0,2016.0,2017.0,2018.0,2020.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1990.0,1991.0,1992.0,1993.0,1996.0,1997.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2013.0,2019.0,2021.0,2022.0,2023.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,2000.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2010.0,2011.0,1988.0,1989.0,1994.0,1995.0,1998.0,1999.0,2008.0,2009.0,2010.0,2011.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2023.0,1990.0,1999.0,2001.0,2002.0,2009.0,2022.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2019.0,2020.0,2021.0,2022.0,2023.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2018.0],
"groupName":["0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","0 - 499","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","500 - 999","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499","1000 - 1499"